In [ ]:
# import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
import random
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import nltk

import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit

# from skimage.transform import resize
# from skimage import io
import numpy as np


# tf.keras.utils.set_random_seed(1234)

MAX_SENTENCE_LEN = 40


2024-02-12 12:53:42.820008: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-12 12:53:42.820038: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-12 12:53:42.821168: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-12 12:53:42.827628: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/santhosh/miniconda3/envs/tf/lib/python3.11/site

In [ ]:
def clean_text(text):

    # remove unnecessary characters in sentences and v

    text = text.lower().strip()
    #Seperate ?.!, with spaces
    text = re.sub(r"([?.!,])", r" \1 ", text)
    #Replace extra spaces with a single space
    text = re.sub(r'[" "]+', " ", text)

    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"there's", "there is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[^a-zA-Z?.!,]+", " ", text)

    #Remove trailing spaces
    text = text.strip()

    return text

In [ ]:
def preprocess(movie_lines, movie_convs, split_ratio, start_tok, end_tok, subword=False):
    #map line ids to line/dialog
    conv_map = {}
    for line in movie_lines:
        if len(line) != 0:
            line_split = line.split(" +++$+++ ")
            conv_map[line_split[0]] = line_split[4]

    #create list containing lists of conversations
    convid_list = []
    for line in movie_convs:
        if len(line) != 0:
            conv = line.split(" +++$+++ ")[-1][1:-1].strip("'").split("', '")
            convid_list.append(conv)

    #split into questions and answers
    input, response  = [], []

    for conv in convid_list:
        for i in range(len(conv)-1):
            input.append(clean_text(conv_map[conv[i]]))
            response.append(clean_text(conv_map[conv[i+1]]))

    #Segregating sentences which habe less than or eqqual to 100 words for faster training
    filtered_input, filtered_response = [], []

    num_qnans_pairs = len(input)

    if not subword:
        for i in range(num_qnans_pairs):
            if len(input[i].split()) <= MAX_SENTENCE_LEN-2 and len(response[i].split()) <= MAX_SENTENCE_LEN-2:
                    filtered_input.append(start_tok + " " + input[i] + " " + end_tok)
                    filtered_response.append(start_tok + " " + response[i] + " " + end_tok)
    else:
        for i in range(num_qnans_pairs):
            if len(input[i].split()) <= MAX_SENTENCE_LEN-2 and len(response[i].split()) <= MAX_SENTENCE_LEN-2:
                    filtered_input.append(input[i])
                    filtered_response.append(response[i])


    #Split to training and test set
    training_size = int(len(filtered_input) * split_ratio)

    #Shuffe the qn answer pairs
    idx = np.arange(len(filtered_input))
    random.shuffle(idx)

    shuffled_input, shuffled_response = [], []

    for i in idx:
        shuffled_input.append(filtered_input[i])
        shuffled_response.append(filtered_response[i])

    train_input, train_responses = shuffled_input[:training_size], shuffled_response[:training_size]
    test_input, test_responses = shuffled_input[training_size:], shuffled_response[training_size:]

    return (train_input, train_responses), (test_input, test_responses)

In [ ]:
def tokenize(train_inputs, train_outputs, test_inputs, test_outputs, oov_tok, num_words):
    if num_words is not None:
        tokenizer = Tokenizer(num_words=num_words, oov_token=oov_tok, lower=False, filters='"#$%&()*+-/:;<=>@[\\]^_`{|}~\t\n',)
    else:
        tokenizer = Tokenizer(oov_token=oov_tok, lower=False)

    tokenizer.fit_on_texts(train_inputs+train_outputs)

    train_input_seq = tokenizer.texts_to_sequences(train_inputs)
    train_output_seq = tokenizer.texts_to_sequences(train_outputs)

    test_input_seq = tokenizer.texts_to_sequences(test_inputs)
    test_output_seq = tokenizer.texts_to_sequences(test_outputs)

    train_input_seq_pad = pad_sequences(train_input_seq, padding="post", maxlen=MAX_SENTENCE_LEN)
    train_output_seq_pad = pad_sequences(train_output_seq, padding="post", maxlen=MAX_SENTENCE_LEN)

    test_input_seq_pad = pad_sequences(test_input_seq, padding="post", maxlen=MAX_SENTENCE_LEN)
    test_output_seq_pad = pad_sequences(test_output_seq, padding="post", maxlen=MAX_SENTENCE_LEN)

    return (train_input_seq_pad, train_output_seq_pad), (test_input_seq_pad, test_output_seq_pad), tokenizer

In [ ]:
def train_dataset(train_in, train_out, batch_size):
    #END token removed from decoder (as there's nothing to predict after the token) input and START token removed from output
    dataset = tf.data.Dataset.from_tensor_slices(({"encoder_in":train_in, "decoder_in":train_out[:,:-1]}, {"outputs": train_out[:, 1:]}))
    dataset = dataset.cache()
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset

In [ ]:
def preprocess_testdata(sentence_list, tokenizer, start_token=None, end_token=None):
    if start_token is not None:
        for sentence in sentence_list:
            sentence = start_token + " " + sentence + " " + end_token

    test_tokens = tokenizer.texts_to_sequences(sentence_list)
    pad_tokens = pad_sequences(test_tokens, padding="post", maxlen=MAX_SENTENCE_LEN)
    return pad_tokens


In [ ]:


#Data preprocessing constants
SPLIT_RATIO = 0.9
START_TOKEN, END_TOKEN = "START", "END"
OOV_TOKEN = "OOV"


#Transformer constants
EMBEDDING_DIM = 128
NUM_LAYERS = 4
NUM_HEADS = 8
UNITS = 512
DROPOUT = 0.1
EPOCHS = 250
NUM_WORDS = None
TRAINING_SIZE = None

# tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

BATCH_SIZE = 32 #* tpu_strategy.num_replicas_in_sync

#open files and save data as variables
with open('./archive/movie_lines.txt', encoding='utf-8', errors='ignore') as f:
    movie_lines = f.read().split('\n')

with open('./archive/movie_conversations.txt', encoding='utf-8', errors='ignore') as f:
    movie_convs = f.read().split('\n')

#preprocess and toknize data
(train_sentences, train_sentences_outputs), (test_sentences, test_outputs) = preprocess(movie_lines, movie_convs, SPLIT_RATIO, START_TOKEN, END_TOKEN, subword=False)

(train_inputs, train_outputs), (test_inputs, test_outputs), tokenizer = tokenize(train_sentences,\
                                                                                    train_sentences_outputs, test_sentences, test_outputs, OOV_TOKEN, NUM_WORDS)


if NUM_WORDS is None:
    vocab_size = len(tokenizer.word_index) + 1

else:
    vocab_size = NUM_WORDS    #for word tokenizer
    #vocab_size = NUM_WORDS + 2    #for subword tokenier



In [ ]:
f = open("chatbot_engine.trt", "rb")
runtime = trt.Runtime(trt.Logger(trt.Logger.WARNING))

engine = runtime.deserialize_cuda_engine(f.read())
context = engine.create_execution_context()

In [ ]:
def predict(ip_batch, pred_tensor, output, d_input1, d_input2, d_output, bindings, stream):
    cuda.memcpy_htod_async(d_input1, ip_batch, stream)
    cuda.memcpy_htod_async(d_input2, pred_tensor, stream)

    context.execute_async_v2(bindings, stream.handle, None)

    cuda.memcpy_dtoh_async(output, d_output, stream)

    stream.synchronize()

    return output

In [ ]:
def prediction_trt(input, query_sentences, tokenizer, start_token, end_token, output=None):

    num_inputs = input.shape[0]

    bleu_list = []
    prediction_list = []
    for _ in range(num_inputs):
        prediction_list.append([start_token])

    prediction_tensor = np.array(prediction_list, dtype=np.int32)

    input = input.astype(np.int32)
    #-------------------------------------------------------
    d_input1 = cuda.mem_alloc(1*input.nbytes)
    #-------------------------------------------------------


    for i in range(MAX_SENTENCE_LEN):
        #-------------------------------------------------------------
        pred_shape = prediction_tensor.shape
        output = np.empty([pred_shape[0], pred_shape[1], vocab_size], dtype=np.int32)
        d_output = cuda.mem_alloc(1*output.nbytes)

        d_input2 = cuda.mem_alloc(prediction_tensor.nbytes)
        bindings = [int(d_input1), int(d_input2), int(d_output)]

        stream = cuda.Stream()
        #-------------------------------------------------------------
        model_out = predict(input, prediction_tensor, output, d_input1, d_input2, d_output, bindings, stream)


        last_words = model_out[:, -1:, :]

        predicted_id = (np.argmax(last_words, axis=-1)).astype(np.int32)

        # concatenated the predicted_id to the output which is given to the decoder
        # as its input.

        prediction_tensor = np.concatenate((prediction_tensor, predicted_id), axis=1)


    for idx, pred in enumerate(prediction_tensor):
        pred_tokens = []
        for token in pred:
            token_np = token #.numpy()
            if(token_np == 0):
                continue
            if token_np != end_token:
                word = tokenizer.index_word[token_np]
                pred_tokens.append(word)
            else:
                break
        pred_sentence = " ".join(pred_tokens[1:])

        query_words = query_sentences[idx].split(" ")
        query = " ".join(query_words)

        # print(f"User: {query}")
        # print(f"Chatbot: {pred_sentence}")

In [ ]:
test_input_sentences = ["Hello, what's up?", "What is your plan?", "Are you going to the gym now?", "When is the book due", "What's the point?", "I'm visiting my parents tomorrow", "It'll be windy next week", "There is a storm tomorrow"] *4
test_inputs = preprocess_testdata(test_input_sentences, tokenizer, START_TOKEN, END_TOKEN)
prediction_trt(test_inputs, test_input_sentences, tokenizer, tokenizer.word_index[START_TOKEN], tokenizer.word_index[END_TOKEN])

In [ ]:
%%timeit

prediction_trt(test_inputs, test_input_sentences, tokenizer, tokenizer.word_index[START_TOKEN], tokenizer.word_index[END_TOKEN])

1.36 s ± 5.09 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
